# Segmenting and Clustering Neighborhoods in Toronto

### Basic imports and installations

In [19]:
!pip install lxml
!pip install bs4
!pip install geocoder
!pip install folium
!pip install geopy
import pandas as pd
import lxml
from bs4 import BeautifulSoup
import io
import geocoder
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


### Page scrapping from wikipedia

In [2]:
!wget -q -O 'neighbor_data.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
df = None  
with open("neighbor_data.html") as fp:
    soup = BeautifulSoup(fp)
    table = soup.find_all('table')[0]
    df = pd.read_html(table.prettify())[0]
    
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


### Dataframe cleaning (we only keep cells with assigned Borough)

df = df[df['Borough'] != 'Not assigned']

df

In [3]:
len(df['Postal Code'].unique())

180

Postal codes are all unique, so no need to merge

### Shape of the dataframe

In [4]:
df.shape

(180, 3)

## Neighborhood coordinates

In [5]:
coord_df = pd.read_csv('https://cocl.us/Geospatial_data')
coord_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


### We integrate the coordinates to our dataframe

In [6]:
df = pd.merge(df,coord_df,on="Postal Code")

In [7]:
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Exploring the neighborhood

### How many neighborhoods were returned

In [8]:
df.groupby('Neighborhood').count()

,Postal Code,Borough,Latitude,Longitude
Neighborhood,,,,
Agincourt,1,1,1,1
"Alderwood, Long Branch",1,1,1,1
"Bathurst Manor, Wilson Heights, Downsview North",1,1,1,1
Bayview Village,1,1,1,1
"Bedford Park, Lawrence Manor East",1,1,1,1
...,...,...,...,...
"Willowdale, Newtonbrook",1,1,1,1
Woburn,1,1,1,1
Woodbine Heights,1,1,1,1


We have 98 different neighborhoods

### Analyze Each Neighborhood

We group the dataframe by neighborhood

In [9]:
df_grouped = df.groupby('Neighborhood').mean().reset_index()
df_grouped

,Neighborhood,Latitude,Longitude
0,Agincourt,43.794200,-79.262029
1,"Alderwood, Long Branch",43.602414,-79.543484
2,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
3,Bayview Village,43.786947,-79.385975
4,"Bedford Park, Lawrence Manor East",43.733283,-79.419750
...,...,...,...
93,"Willowdale, Newtonbrook",43.789053,-79.408493
94,Woburn,43.770992,-79.216917
95,Woodbine Heights,43.695344,-79.318389
96,York Mills West,43.752758,-79.400049


Then we drop the column "Neighborhood" to only keep Latitude and Longtitude

In [10]:
df_grouped_clustering = df_grouped.drop("Neighborhood",axis=1)
df_grouped_clustering

,Latitude,Longitude
0,43.794200,-79.262029
1,43.602414,-79.543484
2,43.754328,-79.442259
3,43.786947,-79.385975
4,43.733283,-79.419750
...,...,...
93,43.789053,-79.408493
94,43.770992,-79.216917
95,43.695344,-79.318389
96,43.752758,-79.400049


# Kmeans clustering

We cluster our data to 5 clusters

In [11]:
n = 5

km = KMeans(n_clusters=n,random_state=21).fit(df_grouped_clustering)
km.labels_

array([4, 0, 2, 2, 2, 3, 1, 3, 1, 3, 3, 0, 4, 3, 3, 3, 1, 1, 3, 3, 2, 0,
       1, 1, 0, 3, 1, 0, 2, 3, 3, 3, 2, 1, 4, 3, 0, 2, 0, 0, 3, 1, 0, 1,
       3, 0, 2, 2, 3, 3, 4, 4, 0, 3, 0, 0, 2, 0, 2, 0, 3, 1, 1, 3, 3, 3,
       3, 2, 4, 0, 0, 4, 0, 3, 3, 1, 3, 3, 3, 3, 1, 3, 0, 1, 3, 3, 4, 1,
       0, 0, 0, 1, 2, 2, 4, 1, 2, 2], dtype=int32)

We add the resulting clusters to our data frame

In [12]:
df_final = df_grouped_clustering.copy()
df_final.insert(0,'Cluster Labels', km.labels_)
df_final.insert(0,'Neighborhood', df_grouped['Neighborhood'])
df_final

,Neighborhood,Cluster Labels,Latitude,Longitude
0,Agincourt,4,43.794200,-79.262029
1,"Alderwood, Long Branch",0,43.602414,-79.543484
2,"Bathurst Manor, Wilson Heights, Downsview North",2,43.754328,-79.442259
3,Bayview Village,2,43.786947,-79.385975
4,"Bedford Park, Lawrence Manor East",2,43.733283,-79.419750
...,...,...,...,...
93,"Willowdale, Newtonbrook",2,43.789053,-79.408493
94,Woburn,4,43.770992,-79.216917
95,Woodbine Heights,1,43.695344,-79.318389
96,York Mills West,2,43.752758,-79.400049


# Clusters visualization

### We get the latitude and longitude values of Toronto

In [13]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [20]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(n)
ys = [i + x + (i*x)**2 for i in range(n)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_final['Latitude'], df_final['Longitude'], df_final['Neighborhood'], df_final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters